In [44]:
import requests
import fitz
import json
import os
import base64

# Enables the display of multiple outputs when running a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [58]:
apikey = os.getenv('OPENAI_API_KEY')

In [59]:
def encode_image(image_path: str):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [60]:
path = 'E:/Downloads/invoice.pdf'

pdf_base64 = encode_image(path)
pdf_base64[:20] + ' ...TRIMMED'

'JVBERi0xLjQKJb662+4K ...TRIMMED'

In [61]:
pdf_bytes = base64.b64decode(pdf_base64)
pdf_bytes

b'%PDF-1.4\n%\xbe\xba\xdb\xee\n1 0 obj\n<</Pages 2 0 R/Type /Catalog/Metadata 10 0 R>>\nendobj\n2 0 obj\n<</Type /Pages/Kids [ 3 0 R ]/Count 1>>\nendobj\n3 0 obj\n<</Type /Page/Parent 2 0 R/Resources <</XObject <</Im0 6 0 R>>/ProcSet 5 0 R>>/MediaBox [ 0 0 782.918 1107.63 ]/CropBox [ 0 0 782.918 1107.63 ]/Contents 4 0 R/Thumb 7 0 R>>\nendobj\n4 0 obj\n<</Length 42/Filter /FlateDecode>>\nstream\nx\xda+\xe42\xb70\xd2\xb34\xb4P0\x00BCC\x03s=3c0;9\x97K\xdf3\xd7@\xc1%\x9f+\x90\x0b\x00\x9b\\\x07\xf4\nendstream\nendobj\n5 0 obj\n[ /PDF /Text /ImageC ]\nendobj\n6 0 obj\n<</Type /XObject/Subtype /Image/Name /Im0/Filter [ /FlateDecode ]/Width 1044/Height 1477/ColorSpace /DeviceRGB/BitsPerComponent 8/SMask 8 0 R/Length 92391>>\nstream\nx\xda\xec\xbdO\xce\xf5:\xd1\xf6\xdbB\xef`x\xc54v\x83\xc6\x91\x98\x00\x88Q $\x9a_\x9b&\x88a\xec\t\xd0<\x1d\xe81\x86\xdd\x84\xbbM\xa4\xa5c\xed\xfa\x9e:\xf5T\xd9\x15\'q\x12g\xad_ti\xe9Z\x89c\x97\xcbN\xd9\x15\xff\xc9\xd7\x7f\xfe\xfb\x05\x00\x00\x00\x00\x00\x00\x00)\x96

In [62]:
doc = fitz.open(stream=pdf_bytes, filetype="pdf")
png_base64_list = []

for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    pix = page.get_pixmap(alpha=True)
    png_bytes = pix.tobytes(output="png")
    png_base64 = base64.b64encode(png_bytes).decode('utf-8')
    png_base64_list.append(png_base64)

png_base64 = png_base64_list[0]
png_base64

'...'

In [63]:
url = 'https://api.openai.com/v1/chat/completions'

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {apikey}'
}

data = {
    "model": "gpt-4-turbo",
    "response_format": { "type": "json_object" },
    "messages": [
    {
        "role": "user",
        "content": [
        {
            "type": "text",
            "text": "Extract the invoice identifier and the total amount without money symbol. Output must be in a json structure with the following keys: identifier and amount."
        },
        {
            "type": "image_url",
            "image_url": {
            "url": f"data:image/jpeg;base64,{png_base64}"
            }
        }
        ]
    }
    ]
}

In [64]:
response = requests.post(url=url, headers=headers, json=data)
response.text

'{\n  "id": "chatcmpl-9hh63QaKMxEQRvhvwX9UrXDK2RYdw",\n  "object": "chat.completion",\n  "created": 1720200795,\n  "model": "gpt-4-turbo-2024-04-09",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": "{\\n  \\"identifier\\": \\"INVO-005\\",\\n  \\"amount\\": \\"425\\"\\n}"\n      },\n      "logprobs": null,\n      "finish_reason": "stop"\n    }\n  ],\n  "usage": {\n    "prompt_tokens": 1140,\n    "completion_tokens": 19,\n    "total_tokens": 1159\n  },\n  "system_fingerprint": "fp_7d35882d38"\n}\n'

In [66]:
json_data = response.json()
json_data

{'id': 'chatcmpl-9hh63QaKMxEQRvhvwX9UrXDK2RYdw',
 'object': 'chat.completion',
 'created': 1720200795,
 'model': 'gpt-4-turbo-2024-04-09',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '{\n  "identifier": "INVO-005",\n  "amount": "425"\n}'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 1140,
  'completion_tokens': 19,
  'total_tokens': 1159},
 'system_fingerprint': 'fp_7d35882d38'}

In [67]:
content = json_data['choices'][0]['message']['content'] if 'choices' in json_data else None
cleaned = json.loads(content)
cleaned

{'identifier': 'INVO-005', 'amount': '425'}